<a href="https://colab.research.google.com/github/mmsamiei/thesis-prototype/blob/master/phase2/distilbert_knowledge_selection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [1]:
!pip install transformers

In [0]:
!cp /content/drive/My\ Drive/Thesis/phase-2/train_tokenized_bert.json ./train.json

In [0]:
from torch.utils.data import Dataset, DataLoader
import os
import torch
import json
from torch.utils.data.sampler import SubsetRandomSampler

In [11]:
import random
random.random()*5

4.570962463792188

In [0]:
import random
class MyDataset(Dataset):
    """My dataset."""

    def __init__(self, json_file):
        """
        Args:
            json_file (string): Path to the json file with annotations.
        """
        self.dialogues = json.load(open(json_file))
        self.root_dir = json_file
        self.dialogues  = [x for x in self.dialogues if x['history'] != None]
        self.dialogues = sorted(self.dialogues, key=lambda x: len(x['history']) + random.random()*5)

    def __len__(self):
        return len(self.dialogues)

    def __getitem__(self, idx):
      
        
        history_lst = self.dialogues[idx]['history']
        true_lst = self.dialogues[idx]['true_sentence']
        false_lst = self.dialogues[idx]['false_sentenc']


        if(len(history_lst)>50):
          history_lst = history_lst[-50:]

        if(len(true_lst)>50):
          true_lst = true_lst[:50]
        
        if(len(false_lst)>50):
          false_lst = false_lst[:50]
        

        history = torch.LongTensor(history_lst)
        true_sample = torch.LongTensor(true_lst)
        false_sample = torch.LongTensor(false_lst)

        sample = {'history': history, 'true_sample': true_sample, 'false_sample': false_sample}

        return sample

In [0]:
dataset = MyDataset('train.json')

In [0]:
def my_collate_fn(batch):

  len_batch = len(batch)

  
  max_len_history = max([len(data['history']) for data in batch])
  max_len_true_sample = max([len(data['true_sample']) for data in batch])
  max_len_false_sample = max([len(data['false_sample']) for data in batch])
  
  padding_ind = 0 ## for bert is 0
  result_history = torch.zeros(len_batch, max_len_history)
  result_true_sample = torch.zeros(len_batch, max_len_true_sample)
  result_false_sample = torch.zeros(len_batch, max_len_false_sample)

  for i, data in enumerate(batch):
    p1 = len(data['history'])
    result_history[i, :p1] = data['history']
    p2 = len(data['true_sample'])
    result_true_sample[i, :p2] = data['true_sample']
    p3 = len(data['false_sample'])
    result_false_sample[i, :p3] = data['false_sample']



  return result_history.long(), result_true_sample.long(), result_false_sample.long()

sampler = torch.utils.data.SequentialSampler(dataset)
dataset_loader = torch.utils.data.DataLoader(dataset, batch_size=64, sampler=sampler,
                                             shuffle=False, collate_fn=my_collate_fn)

In [0]:
for _, batch in enumerate(dataset_loader):
  history, true_sample, false_sample = batch
del history
del true_sample
del false_sample

In [16]:
!nvidia-smi

Fri Jan 24 15:50:17 2020       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 440.44       Driver Version: 418.67       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   38C    P0    26W / 250W |      0MiB / 16280MiB |      0%      Default |
+-------------------------------+----------------------+----------------------+
                                                                               
+-----------------------------------------------------------------------------+
| Processes:                                                       GPU Memory |
|  GPU  

**Model**

In [0]:
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')

In [18]:
import torch
from transformers import *
import numpy as np
import torch.nn as nn

class RepresentModel(nn.Module):
  
  def __init__(self, device):
    super().__init__()

    self.device = device
    
    self.bert = DistilBertModel.from_pretrained('distilbert-base-uncased')

    for p in self.bert.transformer.layer[:-1].parameters():
      p.requires_grad = False

    for p in self.bert.embeddings.parameters():
      p.requires_grad = False


  def forward(self, x):
    ## x = [batch, sent]
    batch_size, sent_len = x.shape[0], x.shape[1]

    temp = self.bert(x)[0]
    ## [batch, sent_len, 768]
    temp = temp[:,0,:]
    return temp

In [19]:
hid_size = 768
model = RepresentModel(device).to(device)

test_len = 20
batch_size = 64
vocab_size = 30000
test_input = torch.LongTensor(batch_size, test_len).random_(1,vocab_size).to(device)
model(test_input).shape

torch.Size([64, 768])

In [20]:
def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

print(f'The model has {count_parameters(model):,} trainable parameters')

The model has 7,087,872 trainable parameters


In [21]:
model

RepresentModel(
  (bert): DistilBertModel(
    (embeddings): Embeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (transformer): Transformer(
      (layer): ModuleList(
        (0): TransformerBlock(
          (dropout): Dropout(p=0.1, inplace=False)
          (attention): MultiHeadSelfAttention(
            (dropout): Dropout(p=0.1, inplace=False)
            (q_lin): Linear(in_features=768, out_features=768, bias=True)
            (k_lin): Linear(in_features=768, out_features=768, bias=True)
            (v_lin): Linear(in_features=768, out_features=768, bias=True)
            (out_lin): Linear(in_features=768, out_features=768, bias=True)
          )
          (sa_layer_norm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
          (ffn): FFN(
            (dropout): Dropout(p=0.1

In [22]:
for p in model.named_parameters():
  print(p[0],"\t",p[1].requires_grad)

bert.embeddings.word_embeddings.weight 	 False
bert.embeddings.position_embeddings.weight 	 False
bert.embeddings.LayerNorm.weight 	 False
bert.embeddings.LayerNorm.bias 	 False
bert.transformer.layer.0.attention.q_lin.weight 	 False
bert.transformer.layer.0.attention.q_lin.bias 	 False
bert.transformer.layer.0.attention.k_lin.weight 	 False
bert.transformer.layer.0.attention.k_lin.bias 	 False
bert.transformer.layer.0.attention.v_lin.weight 	 False
bert.transformer.layer.0.attention.v_lin.bias 	 False
bert.transformer.layer.0.attention.out_lin.weight 	 False
bert.transformer.layer.0.attention.out_lin.bias 	 False
bert.transformer.layer.0.sa_layer_norm.weight 	 False
bert.transformer.layer.0.sa_layer_norm.bias 	 False
bert.transformer.layer.0.ffn.lin1.weight 	 False
bert.transformer.layer.0.ffn.lin1.bias 	 False
bert.transformer.layer.0.ffn.lin2.weight 	 False
bert.transformer.layer.0.ffn.lin2.bias 	 False
bert.transformer.layer.0.output_layer_norm.weight 	 False
bert.transformer.layer

In [0]:
class NoamOpt:
    "Optim wrapper that implements rate."
    def __init__(self, model_size, factor, warmup, optimizer):
        self.optimizer = optimizer
        self._step = 0
        self.warmup = warmup
        self.factor = factor
        self.model_size = model_size
        self._rate = 0
        
    def step(self):
        "Update parameters and rate"
        self._step += 1
        rate = self.rate()
        for p in self.optimizer.param_groups:
            p['lr'] = rate
        self._rate = rate
        self.optimizer.step()
        
    def rate(self, step = None):
        "Implement `lrate` above"
        if step is None:
            step = self._step
        return self.factor * \
            (self.model_size ** (-0.5) *
            min(step ** (-0.5), step * self.warmup ** (-1.5)))
    
    def zero_grad(self):
        self.optimizer.zero_grad()

In [24]:
!pip install -U tqdm

Requirement already up-to-date: tqdm in /usr/local/lib/python3.6/dist-packages (4.41.1)


In [0]:
from tqdm.notebook import tqdm

def train_one_epoch(model,train_iter, optimizer, criterion, clip):
  epoch_loss = 0
  model.train()
  for batch in tqdm(train_iter):
    optimizer.zero_grad()
    history, true_sample, false_sample = batch
    batch_size = history.shape[0]
    ### hist = [batch, sent_len]
    history_rpr = model(history.to(device))
    true_rpr = model(true_sample.to(device))
    false_rpr = model(false_sample.to(device))
    ## rpr = [batch_size, hidden]
    cos = nn.PairwiseDistance().to(device)
    tru_sml = -1*cos(history_rpr, true_rpr)
    fls_sml = -1*cos(history_rpr, false_rpr)
    mini_batch_tensor = torch.ones(batch_size).to(device)
    loss = criterion(tru_sml, fls_sml, mini_batch_tensor)
    loss.backward()
    torch.nn.utils.clip_grad_norm_(model.parameters(), clip)
    optimizer.step()
    epoch_loss += loss.item()
  return epoch_loss / len(train_iter)

In [0]:
def train(model, train_iter, optimizer, criterion, clip, N_EPOCH):
  for epoch in range(N_EPOCH):
    epoch_loss = train_one_epoch(model, train_iter, optimizer, criterion, clip)
    print(epoch_loss)

In [27]:
optimizer = NoamOpt(hid_size, 1, 2000,
              torch.optim.Adam(model.parameters(), lr=0, betas=(0.9, 0.98), eps=1e-9))
criterion = torch.nn.MarginRankingLoss(margin=10).to(device)
train(model, dataset_loader, optimizer, criterion, 1.5, 1)

KeyboardInterrupt: ignored